In [1]:
import pandas as pd

df = pd.read_json("data/dataset/nontrivial_no_longpapers.jsonl", lines=True)
print(f"Length of DataFrame: {len(df)}")
df.head()

Length of DataFrame: 8540


,source_doi,sent_original,sent_no_cit,sent_idx,citation_dois,pubdate,resolved_bibcodes
0,10.1016/j.newar.2024.101694,"Subsequently, Andrews et al. (2017) selected a...","Subsequently, [REF] selected a wide binary can...",58,[10.1093/mnras/stx2000],2024-06-01,[2017MNRAS.472..675A]
1,10.1016/j.newar.2024.101694,Andrews et al. (2017) investigated how the sep...,[REF] investigated how the separation of their...,61,[10.1093/mnras/stx2000],2024-06-01,[2017MNRAS.472..675A]
2,10.1016/j.newar.2024.101694,This led Andrews et al. (2017) to conclude tha...,This led [REF] to conclude that most of the pa...,64,[10.1093/mnras/stx2000],2024-06-01,[2017MNRAS.472..675A]
3,10.1016/j.newar.2024.101694,It may also owe in part to the mass ratio dist...,It may also owe in part to the mass ratio dist...,90,[10.1093/mnras/stz2480],2024-06-01,[2019MNRAS.489.5822E]
4,10.1016/j.newar.2024.101694,Hwang et al. (2022c) used a related method to ...,[REF] used a related method to study the eccen...,110,[10.3847/2041-8213/ac7c70],2024-06-01,[2022ApJ...933L..32H]


In [2]:
# Get the unique dois from the database contribution table
from database.database import Database

db = Database()
db.test_connection()

=================================CONFIG=================================
Database         User             Host                             Port            
citelinedb       bbasseri         localhost                        5432            
Database version: ('PostgreSQL 17.5 (Homebrew) on aarch64-apple-darwin24.4.0, compiled by Apple clang version 17.0.0 (clang-1700.0.13.3), 64-bit',)


In [3]:
from Embedders import get_embedder

embedder = get_embedder(model_name="BAAI/bge-large-en-v1.5", device="mps", normalize=True)

In [ ]:
from database.database import VectorQueryResult
def get_results(example: pd.Series) -> list[VectorQueryResult]:
    """
    Get the results for a given example.
    """
    query = example['sent_no_cit']
    query_vector = embedder([query])[0]
    results = db.vector_search(
        query_vector=query_vector,
        target_table='contributions',
        target_column='embedding',
        metric='vector_cosine_ops',
        pubdate=example['pubdate'],
        top_k=5,
    )
    return results

In [33]:
for i in range(100):
    print(i, end=' ')
    if (i+1) % 40 == 0:
        print()
    example = df.iloc[i]
    results = get_results(example)
    result_dois = {result.doi for result in results}
    closest_distance = results[0].distance
    if closest_distance > 0.25:
        print(f"Target doi are: {example['citation_dois']}")
        print(f"Retrievied doi are: {result_dois}")
        print(f"Intersection size: {len(result_dois & set(example['citation_dois']))}")
        pprint(f"Query: {example['sent_no_cit']}")
        for result in results:
            pprint(result)
        print("==========================")

0 1 Target doi are: ['10.1093/mnras/stx2000']
Retrievied doi are: {'10.1088/1742-6596/363/1/012035', '10.1086/303547', '10.1093/mnras/stz960', '10.1086/522073', '10.1086/523887'}
Intersection size: 0
('Query: [REF] investigated how the separation of their candidate pairs varied '
 'with distance and astrometric uncertainties.')
VectorSearchResult(text='The study highlights the degeneracies between the '
                        'power-law slopes of the mass ratio distribution (α) '
                        'and the orbital separation distribution (β), and the '
                        'sensitivity of the best-fitting solutions to the '
                        'adopted boundaries of these distributions.',
                   doi='10.1086/522073',
                   pubdate=datetime.date(2007, 11, 1),
                   distance=0.2555269267994602)
VectorSearchResult(text='Key species in protoplanetary disks can be separated '
                        'into two groups based on the sensitivit

Target dois are: ['10.1093/mnras/stx2000']


In [23]:
from pprint import pprint
pprint(f"Query sentence: {df.iloc[200]['sent_no_cit']}")
pprint(results)

('Query sentence: The points show the values given in table 4 of [REF] , '
 'computed for a Salpeter IMF in the 1−100M ⊙ range, and divided by a mass '
 'conversion factor of 2.55 to rescale to a mass range 0.1−100M ⊙ .')
[VectorSearchResult(text='Compared stellar mass-to-light ratios using standard '
                         'and scaled Salpeter IMFs, finding the scaled IMF '
                         'more realistic and indicating lower stellar mass '
                         'fractions, especially for early-type galaxies.',
                    doi='10.1086/496913',
                    pubdate=datetime.date(2005, 12, 1),
                    distance=0.2721689613217202),
 VectorSearchResult(text='The slope of the IMF at larger masses is equal to '
                         '3/(4-β), where β is the inertial range spectral '
                         'index of supersonic turbulence, yielding a slope of '
                         '1.33 for β=1.74, closely matching the Salpeter IMF '
       